# Initial Design

This notebook contains the code used to select the reaction conditions tested for initial training of the GP. A modified latin hypercube sampling (LHS) strategy is used to select both discrete and continuous variables.  We previously demonstrated that several different designs work well for solvent selection, so LHS was chosen since it is already implemented by GPyOpt.  

## 1. Setup

Let's get everything loaded and ready to go.

In [2]:
#Autoreload automatically reloads any depdencies as you change them
%load_ext autoreload
%autoreload 2

In [3]:
#Import all the necessary packages
from summit.data import solvent_ds, ucb_list
from summit.domain import Domain, ContinuousVariable, DiscreteVariable, DescriptorsVariable
from summit.experiment_design import LatinDesign
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [4]:
missed_cas_numbers = []
successful_cas_numbers = []
for i, index in enumerate(ucb_list):
    try:
        select = solvent_ds.xs(index, level='cas_number', drop_level=False)
        successful_cas_numbers.append(index)
    except KeyError:
        missed_cas_numbers.append(index)
print(f"{len(missed_cas_numbers)} out of {len(ucb_list)} solvents from the UCB list are not in our solvent database.")

36 out of 115 solvents from the UCB list are not in our solvent database.


In [5]:
#Specify the optimization space

domain = Domain()

domain += ContinuousVariable(name='temperature',
                             description = "reaction temperature",
                             bounds=[20, 50])

domain += ContinuousVariable(name="acid_conc",
                             description = "propionic acid concentration",
                             bounds=[1,40])

domain += ContinuousVariable(name="cat_load",
                             description = "catalyst loading",
                             bounds=[0.1, 10])

domain += ContinuousVariable(name="co_cat_load",
                             description = "co-catalyst loading",
                             bounds=[15, 1500])

domain += ContinuousVariable(name="acrylate_amine_ratio",
                             description = "molar ratio of acrylate to amine",
                             bounds = [0.8, 2])


domain += ContinuousVariable(name="aldehyde_amine_ratio",
                             description = "molar ratio of aldehyde to amine",
                             bounds=[0.8, 2])

domain += DiscreteVariable(name="co_cat",
                           description="enumeration of the two potential cocatalysts",
                           levels = ['co_cat_1', 'co_cat_2'])


domain += DescriptorsVariable(name="solvent",
                             description="17 descriptors of the solvent",
                             df=solvent_ds,
                             select_subset=successful_cas_numbers,
                             select_index='cas_number')

domain #The domain should display as an html table 

Name,Type,Description,Values
temperature,continuous,reaction temperature,"[20,50]"
acid_conc,continuous,propionic acid concentration,"[1,40]"
cat_load,continuous,catalyst loading,"[0.1,10]"
co_cat_load,continuous,co-catalyst loading,"[15,1500]"
acrylate_amine_ratio,continuous,molar ratio of acrylate to amine,"[0.8,2]"
aldehyde_amine_ratio,continuous,molar ratio of aldehyde to amine,"[0.8,2]"
co_cat,discrete,enumeration of the two potential cocatalysts,2 levels
solvent,descriptors,17 descriptors of the solvent,459 examples of 17 descriptors


## 2. Construct Initial Design

In [1]:
lhs = LatinDesign(domain)
experiments = lhs.generate_experiments(20)
experiments

NameError: name 'LatinDesign' is not defined

In [7]:
expr_df = experiments.to_frame()
expr_df.to_csv('initial_experiments.csv')

In [2]:
x = [1,1,2]